In [2]:
import requests, xmltodict, pandas as pd
from pandas import json_normalize
from zeep import Client, helpers
from datetime import datetime
import ast

In [9]:
def listar_ops(wsdl_url):
    c = Client(wsdl=wsdl_url)
    for svc in c.wsdl.services.values():
        for port in svc.ports.values():
            binding = port.binding
            for name, op in binding._operations.items():
                # Nombre de la operación
                args = []
                if op.input and op.input.body and op.input.body.type:
                    args = [elt[0] for elt in op.input.body.type.elements]  # nombres de parámetros
                print(f"{name}({', '.join(args)})")


listar_ops("https://opendata.camara.cl/camaradiputados/WServices/WSProyectosAcuerdo.asmx?WSDL")

retornarProyectosAcuerdoXAnno(prmAnno)
retornarProyectoAcuerdo(prmProyectoAcuerdoId)
retornarProyectosAcuerdoXAnno(prmAnno)
retornarProyectoAcuerdo(prmProyectoAcuerdoId)
retornarProyectosAcuerdoXAnno(prmAnno)
retornarProyectoAcuerdo(prmProyectoAcuerdoId)
retornarProyectosAcuerdoXAnno(prmAnno)
retornarProyectoAcuerdo(prmProyectoAcuerdoId)


In [5]:
def safe_serialize(obj):
    # tipos base
    if obj is None or isinstance(obj, (str, int, float, bool)):
        return obj
    if isinstance(obj, list):
        return [safe_serialize(x) for x in obj]
    if isinstance(obj, dict):
        return {k: safe_serialize(v) for k, v in obj.items()}

    # intentar con zeep.helpers.serialize_object (si está disponible)
    try:
        from zeep.helpers import serialize_object
        ser = serialize_object(obj)
        if ser is not None:
            return safe_serialize(ser)  # recursivo por si trae más objetos
    except Exception:
        pass

    # objetos Zeep suelen tener __values__
    vals = getattr(obj, "__values__", None)
    if isinstance(vals, dict):
        return {k: safe_serialize(v) for k, v in vals.items()}

    # fallback genérico
    try:
        return {k: safe_serialize(v) for k, v in vars(obj).items()}
    except Exception:
        return obj

In [14]:
def get_proyecto_year(year):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSProyectosAcuerdo.asmx?WSDL")
    res = c.service.retornarProyectosAcuerdoXAnno(year)
    if res:
        d = safe_serialize(res) or {}
        df = json_normalize(d)
        return df

In [15]:
df_proyectos_year = pd.concat((get_proyecto_year(year) for year in range(1990, 2026)))

In [17]:
df_proyectos_year.to_csv("proyectos_year.csv",  index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")

### Proyecto de Acuerdo

In [22]:
def get_proyecto(idx):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSProyectosAcuerdo.asmx?WSDL")
    res = c.service.retornarProyectoAcuerdo(idx)
    if res:
        d = safe_serialize(res) or {}
        del d["Autores"]
        del d["Periodo"]
        del d["OficiosEnviados"]
        del d["RespuestasRecibidas"]
        df = json_normalize(d)
        return df

In [24]:
df_proyectos = pd.concat((get_proyecto(idx) for idx in df_proyectos_year["Id"]))

In [27]:
df_proyectos.to_csv("proyectos.csv", index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")

In [45]:
def get_autores(idx):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSProyectosAcuerdo.asmx?WSDL")
    res = c.service.retornarProyectoAcuerdo(idx)
    if res:
        d = safe_serialize(res)
        autores = safe_serialize(d["Autores"]["ParlamentarioAutor"])
        df = json_normalize(autores)
        df["proyecto_id"] = d["Id"]
        return df
def get_oficios(idx):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSProyectosAcuerdo.asmx?WSDL")
    res = c.service.retornarProyectoAcuerdo(idx)
    if res:
        d = safe_serialize(res)
        if d["OficiosEnviados"]:
            oficio = safe_serialize(d["OficiosEnviados"]["Oficio"])
            df = json_normalize(oficio)
            df["proyecto_id"] = d["Id"]
            return df
def get_respuestas(idx):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSProyectosAcuerdo.asmx?WSDL")
    res = c.service.retornarProyectoAcuerdo(idx)
    if res:
        d = safe_serialize(res)
        if d["RespuestasRecibidas"]:
            respuesta = safe_serialize(d["RespuestasRecibidas"]["Respuesta"])  
            df = json_normalize(respuesta)
            df["proyecto_id"] = d["Id"]
            return df

In [38]:
df_autores = pd.concat((get_autores(idx) for idx in df_proyectos_year["Id"]))
df_oficios = pd.concat((get_oficios(idx) for idx in df_proyectos_year["Id"]))
df_respuestas = pd.concat((get_respuestas(idx) for idx in df_proyectos_year["Id"]))

In [50]:
df_oficios.to_csv("oficios_proyectos.csv", index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")
df_respuestas.to_csv("respuestas_proyectos.csv", index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")
df_autores.to_csv("autores_proyectos.csv", index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")